In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report,accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.feature_selection import chi2,SelectKBest
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [5]:
df=pd.read_csv('steam_train.csv')
df1=pd.read_csv('steam_test.csv')
y=df['price']
df.head()
df.drop(['name','appid'],inplace=True,axis=1)
df1.drop(['name','appid'],inplace=True,axis=1)

In [6]:
df

,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,2017-09-28,1,DysTop,DysTop,windows,0,Single-player;Partial Controller Support,Action;Indie,Action;Indie;Space,0,2,2,0,0,0-20000,2.89
1,2017-06-29,1,Kodansha;Polygon Pictures Inc.;Lantis,Degica,windows,0,Single-player;Partial Controller Support,Casual,Casual;VR;Anime,0,30,4,0,0,0-20000,3.99
2,2017-09-07,1,Matt Bitner Games,Matt Bitner Games,windows;mac;linux,0,Single-player;Local Co-op;Shared/Split Screen;...,Violent;Gore;Action;Indie,Action;Metroidvania;Indie,60,271,30,279,279,20000-50000,10.29
3,2017-02-15,1,ASTRO PORT,Nyu Media,windows;linux,0,Single-player;Steam Achievements;Steam Trading...,Action;Indie,Action;Indie;Shoot 'Em Up,53,16,0,0,0,0-20000,4.79
4,2019-02-19,1,Bony Yousuf,The Quantum Astrophysicists Guild,windows;mac;linux,0,Single-player;Steam Achievements;Full controll...,Action;Casual;Indie;Strategy,Indie;Action;Casual,24,29,1,0,0,0-20000,7.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25039,2018-04-24,1,Geckoo1337,Geckoo1337,windows,0,Single-player;Steam Achievements,Casual;Indie,Casual;Indie;Puzzle,9,15,0,0,0,0-20000,0.79
25040,2015-05-13,1,Playsaurus,Playsaurus,windows;mac,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Casual;Free to Play;Indie;RPG;Simula...,Clicker;Free to Play;Casual,111,42530,4734,9742,463,5000000-10000000,0.00
25041,2011-07-15,1,Headup Games / Crenetic,Headup Games,windows,0,Single-player;Multi-player;Co-op;Steam Trading...,Action;Strategy,Zombies;Strategy;Action,0,258,410,142,157,100000-200000,3.99
25042,2017-10-24,1,Atomic Wolf,Atomic Wolf,windows,0,Single-player;Steam Achievements;Partial Contr...,Action;Casual;Indie,Indie;Action;Casual,36,48,8,0,0,0-20000,3.99


# Uniquely identifying numerical and categorical features

In [7]:
numeric_feat=[feat for feat in df.columns if df[feat].dtypes!='O']
categor_feat=[feat for feat in df.columns if df[feat].dtypes=='O']

In [8]:
numeric_feat

['english',
 'required_age',
 'achievements',
 'positive_ratings',
 'negative_ratings',
 'average_playtime',
 'median_playtime',
 'price']

In [9]:
numeric_feat.remove('english')
numeric_feat.remove('required_age')
numeric_feat.remove('price') 
numeric_feat

['achievements',
 'positive_ratings',
 'negative_ratings',
 'average_playtime',
 'median_playtime']

In [10]:
categor_feat

['release_date',
 'developer',
 'publisher',
 'platforms',
 'categories',
 'genres',
 'steamspy_tags',
 'owners']

In [11]:
df.head()

,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,2017-09-28,1,DysTop,DysTop,windows,0,Single-player;Partial Controller Support,Action;Indie,Action;Indie;Space,0,2,2,0,0,0-20000,2.89
1,2017-06-29,1,Kodansha;Polygon Pictures Inc.;Lantis,Degica,windows,0,Single-player;Partial Controller Support,Casual,Casual;VR;Anime,0,30,4,0,0,0-20000,3.99
2,2017-09-07,1,Matt Bitner Games,Matt Bitner Games,windows;mac;linux,0,Single-player;Local Co-op;Shared/Split Screen;...,Violent;Gore;Action;Indie,Action;Metroidvania;Indie,60,271,30,279,279,20000-50000,10.29
3,2017-02-15,1,ASTRO PORT,Nyu Media,windows;linux,0,Single-player;Steam Achievements;Steam Trading...,Action;Indie,Action;Indie;Shoot 'Em Up,53,16,0,0,0,0-20000,4.79
4,2019-02-19,1,Bony Yousuf,The Quantum Astrophysicists Guild,windows;mac;linux,0,Single-player;Steam Achievements;Full controll...,Action;Casual;Indie;Strategy,Indie;Action;Casual,24,29,1,0,0,0-20000,7.19


In [12]:
df1.head()

,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners
0,2016-08-18,1,Luke Cripps,Fellowplayer,windows,0,Single-player,Indie,Indie;Horror,14,6,3,0,0,0-20000
1,2019-04-03,1,RockyDev,RockyDev,windows,0,Single-player;Multi-player;Co-op;Full controll...,Action;Early Access,Early Access;Action;Violent,0,0,1,88,88,0-20000
2,2014-06-05,1,Owl Cave,Owl Cave,windows,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Indie,Adventure;Indie;Point & Click,21,264,99,332,332,50000-100000
3,2013-01-09,1,SkyGoblin,SkyGoblin,windows;mac;linux,0,Single-player;Steam Trading Cards,Adventure;Indie,Adventure;Point & Click;Indie,0,902,129,303,327,200000-500000
4,2018-07-02,1,Zerstoren Games,Zerstoren Games,windows,0,Single-player;Partial Controller Support,Action;Adventure;Indie;Simulation;Strategy;Ear...,Early Access;Simulation;Indie,0,6,11,0,0,0-20000


In [13]:
df.describe()

,english,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price
count,25044.000000,25044.000000,25044.000000,2.504400e+04,25044.000000,25044.000000,25044.000000,25044.000000
mean,0.981113,0.355574,45.303865,1.004463e+03,215.739818,146.385202,139.477599,6.055217
std,0.136128,2.408379,350.998233,1.956267e+04,4447.824914,1783.038628,2114.615241,7.284552
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,6.000000e+00,2.000000,0.000000,0.000000,1.690000
50%,1.000000,0.000000,7.000000,2.400000e+01,9.000000,0.000000,0.000000,3.990000
75%,1.000000,0.000000,23.000000,1.240000e+02,41.000000,0.000000,0.000000,7.190000
max,1.000000,18.000000,5394.000000,2.644404e+06,487076.000000,190625.000000,190625.000000,209.990000


In [14]:
df1.describe()

,english,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime
count,2031.000000,2031.000000,2031.000000,2031.000000,2031.000000,2031.000000,2031.000000
mean,0.981290,0.346627,44.570655,952.408666,152.915805,191.973412,227.173806
std,0.135532,2.377644,372.762350,9369.513578,904.502089,2301.593278,4327.346725
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,7.000000,2.000000,0.000000,0.000000
50%,1.000000,0.000000,8.000000,25.000000,10.000000,0.000000,0.000000
75%,1.000000,0.000000,24.000000,153.000000,46.000000,1.000000,1.000000
max,1.000000,18.000000,9821.000000,363721.000000,25607.000000,95242.000000,190445.000000


# Handling outliers, by setting the threshold value as the 3rd quantile, and thereby replacing all values outside the third quantile with the values in the third quantile

In [15]:
for i in numeric_feat:
    ub=df[i].quantile(0.99)
    lb=df[i].quantile(0.01)
    df.loc[df[i]>=ub,i]=df[i].mean()
    df.loc[df[i]<=lb,i]=df[i].mean()
   
for i in numeric_feat:
    ub=df1[i].quantile(0.99)
    lb=df1[i].quantile(0.01)
    df1.loc[df1[i]>=ub,i]=df1[i].mean()
    df1.loc[df1[i]<=lb,i]=df1[i].mean()

In [16]:
df.describe()

,english,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price
count,25044.000000,25044.000000,25044.000000,25044.000000,25044.000000,25044.000000,25044.000000,25044.000000
mean,0.981113,0.355574,23.518396,363.561831,89.764508,114.530204,110.062438,6.055217
std,0.136128,2.408379,24.257217,1271.840343,252.826786,186.338997,167.758179,7.284552
min,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,1.000000,0.000000,16.349156,7.000000,4.000000,64.557606,62.039246,1.690000
50%,1.000000,0.000000,16.349156,28.000000,16.000000,64.557606,62.039246,3.990000
75%,1.000000,0.000000,23.000000,155.000000,81.043381,64.557606,62.039246,7.190000
max,1.000000,18.000000,337.000000,14966.000000,2796.000000,2098.000000,1777.000000,209.990000


In [17]:
df1.describe()

,english,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime
count,2031.000000,2031.000000,2031.000000,2031.000000,2031.000000,2031.000000,2031.000000
mean,0.981290,0.346627,23.378359,399.208047,93.919889,127.052854,121.393634
std,0.135532,2.377644,23.225222,1373.185812,250.543221,217.209089,183.855373
min,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,0.000000,16.299844,8.000000,4.000000,72.688548,69.450837
50%,1.000000,0.000000,16.299844,29.000000,17.000000,72.688548,69.450837
75%,1.000000,0.000000,24.000000,187.000000,85.080863,72.688548,69.450837
max,1.000000,18.000000,348.000000,17273.000000,2606.000000,2901.000000,2286.000000


In [18]:
df

,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,2017-09-28,1,DysTop,DysTop,windows,0,Single-player;Partial Controller Support,Action;Indie,Action;Indie;Space,16.349156,2.000000,2.000000,64.557606,62.039246,0-20000,2.89
1,2017-06-29,1,Kodansha;Polygon Pictures Inc.;Lantis,Degica,windows,0,Single-player;Partial Controller Support,Casual,Casual;VR;Anime,16.349156,30.000000,4.000000,64.557606,62.039246,0-20000,3.99
2,2017-09-07,1,Matt Bitner Games,Matt Bitner Games,windows;mac;linux,0,Single-player;Local Co-op;Shared/Split Screen;...,Violent;Gore;Action;Indie,Action;Metroidvania;Indie,60.000000,271.000000,30.000000,279.000000,279.000000,20000-50000,10.29
3,2017-02-15,1,ASTRO PORT,Nyu Media,windows;linux,0,Single-player;Steam Achievements;Steam Trading...,Action;Indie,Action;Indie;Shoot 'Em Up,53.000000,16.000000,81.043381,64.557606,62.039246,0-20000,4.79
4,2019-02-19,1,Bony Yousuf,The Quantum Astrophysicists Guild,windows;mac;linux,0,Single-player;Steam Achievements;Full controll...,Action;Casual;Indie;Strategy,Indie;Action;Casual,24.000000,29.000000,1.000000,64.557606,62.039246,0-20000,7.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25039,2018-04-24,1,Geckoo1337,Geckoo1337,windows,0,Single-player;Steam Achievements,Casual;Indie,Casual;Indie;Puzzle,9.000000,15.000000,81.043381,64.557606,62.039246,0-20000,0.79
25040,2015-05-13,1,Playsaurus,Playsaurus,windows;mac,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Casual;Free to Play;Indie;RPG;Simula...,Clicker;Free to Play;Casual,111.000000,1004.463345,215.739818,146.385202,463.000000,5000000-10000000,0.00
25041,2011-07-15,1,Headup Games / Crenetic,Headup Games,windows,0,Single-player;Multi-player;Co-op;Steam Trading...,Action;Strategy,Zombies;Strategy;Action,16.349156,258.000000,410.000000,142.000000,157.000000,100000-200000,3.99
25042,2017-10-24,1,Atomic Wolf,Atomic Wolf,windows,0,Single-player;Steam Achievements;Partial Contr...,Action;Casual;Indie,Indie;Action;Casual,36.000000,48.000000,8.000000,64.557606,62.039246,0-20000,3.99


# Using mean endocing to create a dictionary and mapping it to al categorical variables

In [19]:

for i in categor_feat:
    mean_ord=df.groupby([i])['price'].mean().to_dict()
    df[i]=df[i].map(mean_ord)
    df1[i]=df1[i].map(mean_ord)

In [20]:
df

,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,7.334000,1,2.890000,2.890000,5.987534,0,4.801823,4.978147,5.925000,16.349156,2.000000,2.000000,64.557606,62.039246,5.586209,2.89
1,4.336667,1,4.323333,11.744167,5.987534,0,4.801823,4.631561,3.990000,16.349156,30.000000,4.000000,64.557606,62.039246,5.586209,3.99
2,4.577500,1,10.290000,10.290000,6.507862,0,10.290000,4.821667,10.290000,60.000000,271.000000,30.000000,279.000000,279.000000,6.212720,10.29
3,4.364444,1,4.561429,5.167778,4.950505,0,7.700870,4.978147,4.799011,53.000000,16.000000,81.043381,64.557606,62.039246,5.586209,4.79
4,5.887200,1,7.190000,11.056667,6.507862,0,11.060607,4.448519,3.286333,24.000000,29.000000,1.000000,64.557606,62.039246,5.586209,7.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25039,5.720263,1,0.790000,0.790000,5.987534,0,3.946080,2.560707,2.920676,9.000000,15.000000,81.043381,64.557606,62.039246,5.586209,0.79
25040,6.560000,1,11.895000,9.926667,6.003535,0,0.000000,0.000000,0.000000,111.000000,1004.463345,215.739818,146.385202,463.000000,11.007381,0.00
25041,4.390000,1,3.990000,3.990000,5.987534,0,11.390000,8.363521,5.490000,16.349156,258.000000,410.000000,142.000000,157.000000,7.210188,3.99
25042,7.471333,1,3.990000,3.990000,5.987534,0,7.369227,3.322390,3.286333,36.000000,48.000000,8.000000,64.557606,62.039246,5.586209,3.99


In [21]:
df

,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,7.334000,1,2.890000,2.890000,5.987534,0,4.801823,4.978147,5.925000,16.349156,2.000000,2.000000,64.557606,62.039246,5.586209,2.89
1,4.336667,1,4.323333,11.744167,5.987534,0,4.801823,4.631561,3.990000,16.349156,30.000000,4.000000,64.557606,62.039246,5.586209,3.99
2,4.577500,1,10.290000,10.290000,6.507862,0,10.290000,4.821667,10.290000,60.000000,271.000000,30.000000,279.000000,279.000000,6.212720,10.29
3,4.364444,1,4.561429,5.167778,4.950505,0,7.700870,4.978147,4.799011,53.000000,16.000000,81.043381,64.557606,62.039246,5.586209,4.79
4,5.887200,1,7.190000,11.056667,6.507862,0,11.060607,4.448519,3.286333,24.000000,29.000000,1.000000,64.557606,62.039246,5.586209,7.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25039,5.720263,1,0.790000,0.790000,5.987534,0,3.946080,2.560707,2.920676,9.000000,15.000000,81.043381,64.557606,62.039246,5.586209,0.79
25040,6.560000,1,11.895000,9.926667,6.003535,0,0.000000,0.000000,0.000000,111.000000,1004.463345,215.739818,146.385202,463.000000,11.007381,0.00
25041,4.390000,1,3.990000,3.990000,5.987534,0,11.390000,8.363521,5.490000,16.349156,258.000000,410.000000,142.000000,157.000000,7.210188,3.99
25042,7.471333,1,3.990000,3.990000,5.987534,0,7.369227,3.322390,3.286333,36.000000,48.000000,8.000000,64.557606,62.039246,5.586209,3.99


In [22]:
df1=df1.fillna(df1.mean())
df1

,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners
0,7.059375,1,1.990000,1.990000,5.987534,0,4.504338,3.971828,2.415000,14.000000,6.000000,3.0,72.688548,69.450837,5.586209
1,5.758947,1,6.512346,6.551533,5.987534,0,5.862759,9.156000,6.602000,16.299844,390.554693,1.0,88.000000,88.000000,5.586209
2,6.856667,1,4.790000,4.790000,5.987534,0,7.743807,5.851489,6.319912,21.000000,264.000000,99.0,332.000000,332.000000,6.247660
3,7.190000,1,12.823333,14.990000,6.507862,0,4.272685,5.851489,7.528095,16.299844,902.000000,129.0,303.000000,327.000000,7.220638
4,3.172727,1,6.512346,6.551533,5.987534,0,4.801823,10.256667,8.543404,16.299844,6.000000,11.0,72.688548,69.450837,5.586209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026,4.734444,1,10.906667,9.640000,5.987534,0,10.081282,12.360457,5.584607,20.000000,746.000000,149.0,222.000000,360.000000,6.247660
2027,6.888056,1,4.523333,4.523333,6.003535,0,3.946080,6.878182,7.960882,15.000000,7.000000,8.0,72.688548,69.450837,5.586209
2028,4.890278,1,6.512346,5.421707,6.003535,0,4.504338,8.764680,5.356667,16.299844,4.000000,6.0,72.688548,69.450837,5.586209
2029,4.444706,1,7.190000,6.551533,5.987534,0,4.542500,6.823333,4.523333,16.299844,5.000000,2.0,72.688548,69.450837,5.586209


In [23]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x=x.astype('int')
y=y.astype('int')

# Using Chi square test to select top 10 features

In [24]:
best_feat=SelectKBest(score_func=chi2,k=10)
fit=best_feat.fit(x,y)
dfscore=pd.DataFrame(fit.scores_)
dfcols=pd.DataFrame(x.columns)
feat_score=pd.concat([dfcols,dfscore],axis=1)
feat_score.columns=['Specs','Score']
feat_score
print(feat_score.nlargest(10,'Score'))

               Specs         Score
10  positive_ratings  8.780943e+06
11  negative_ratings  1.658536e+06
12  average_playtime  6.127696e+05
13   median_playtime  4.604511e+05
2          developer  1.620451e+05
3          publisher  1.329251e+05
8      steamspy_tags  9.694053e+04
6         categories  2.859210e+04
7             genres  2.744814e+04
9       achievements  2.194626e+04


In [25]:
df=df[['positive_ratings','negative_ratings','publisher','developer','average_playtime','median_playtime','price']]

In [26]:
df1=df1[['positive_ratings','negative_ratings','publisher','developer','average_playtime','median_playtime']]

In [27]:
df

,positive_ratings,negative_ratings,publisher,developer,average_playtime,median_playtime,price
0,2.000000,2.000000,2.890000,2.890000,64.557606,62.039246,2.89
1,30.000000,4.000000,11.744167,4.323333,64.557606,62.039246,3.99
2,271.000000,30.000000,10.290000,10.290000,279.000000,279.000000,10.29
3,16.000000,81.043381,5.167778,4.561429,64.557606,62.039246,4.79
4,29.000000,1.000000,11.056667,7.190000,64.557606,62.039246,7.19
...,...,...,...,...,...,...,...
25039,15.000000,81.043381,0.790000,0.790000,64.557606,62.039246,0.79
25040,1004.463345,215.739818,9.926667,11.895000,146.385202,463.000000,0.00
25041,258.000000,410.000000,3.990000,3.990000,142.000000,157.000000,3.99
25042,48.000000,8.000000,3.990000,3.990000,64.557606,62.039246,3.99


In [28]:
df1

,positive_ratings,negative_ratings,publisher,developer,average_playtime,median_playtime
0,6.000000,3.0,1.990000,1.990000,72.688548,69.450837
1,390.554693,1.0,6.551533,6.512346,88.000000,88.000000
2,264.000000,99.0,4.790000,4.790000,332.000000,332.000000
3,902.000000,129.0,14.990000,12.823333,303.000000,327.000000
4,6.000000,11.0,6.551533,6.512346,72.688548,69.450837
...,...,...,...,...,...,...
2026,746.000000,149.0,9.640000,10.906667,222.000000,360.000000
2027,7.000000,8.0,4.523333,4.523333,72.688548,69.450837
2028,4.000000,6.0,5.421707,6.512346,72.688548,69.450837
2029,5.000000,2.0,6.551533,7.190000,72.688548,69.450837


In [29]:
from sklearn.metrics import SCORERS
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [30]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [31]:
x_train

,positive_ratings,negative_ratings,publisher,developer,average_playtime,median_playtime
6655,30.0,19.000000,8.740417,10.365000,64.557606,62.039246
12223,173.0,26.000000,4.290000,4.290000,15.000000,15.000000
24556,83.0,45.000000,0.790000,0.790000,64.557606,62.039246
7209,73.0,16.000000,3.232857,2.090000,64.557606,62.039246
3510,5.0,7.000000,2.923333,2.923333,64.557606,62.039246
...,...,...,...,...,...,...
15797,86.0,5.000000,8.151905,6.990000,64.557606,62.039246
23610,492.0,264.000000,12.669487,13.418571,17.000000,17.000000
18455,77.0,13.000000,5.516316,3.990000,162.000000,162.000000
9128,3.0,81.043381,5.790000,5.790000,64.557606,62.039246


In [32]:
df

,positive_ratings,negative_ratings,publisher,developer,average_playtime,median_playtime,price
0,2.000000,2.000000,2.890000,2.890000,64.557606,62.039246,2.89
1,30.000000,4.000000,11.744167,4.323333,64.557606,62.039246,3.99
2,271.000000,30.000000,10.290000,10.290000,279.000000,279.000000,10.29
3,16.000000,81.043381,5.167778,4.561429,64.557606,62.039246,4.79
4,29.000000,1.000000,11.056667,7.190000,64.557606,62.039246,7.19
...,...,...,...,...,...,...,...
25039,15.000000,81.043381,0.790000,0.790000,64.557606,62.039246,0.79
25040,1004.463345,215.739818,9.926667,11.895000,146.385202,463.000000,0.00
25041,258.000000,410.000000,3.990000,3.990000,142.000000,157.000000,3.99
25042,48.000000,8.000000,3.990000,3.990000,64.557606,62.039246,3.99


In [33]:
clf=RandomForestRegressor()

In [34]:
clf.fit(x_train,y_train)

RandomForestRegressor()

In [35]:
y_pred=clf.predict(x_test)

In [36]:
mean_squared_error(y_pred,y_test)

11.153616967492406

In [37]:
y_pred=clf.predict(df1)

In [39]:
df=pd.read_csv('steam_test.csv')
new=pd.DataFrame(df['appid'],columns=['appid'])
new['price']=y_pred
new.to_csv('Sub.csv',index=False)

In [35]:
x_train.shape

(17530, 7)

# Just for fun tried implementing an ANN, didnt help with accuracy though

In [47]:
from tensorflow.keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU,PReLU,ELU
from tensorflow.keras.layers import Dropout


# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(50, kernel_initializer = 'he_uniform',activation='relu',input_dim = 7))

# Adding the second hidden layer
classifier.add(Dense(25, kernel_initializer = 'he_uniform',activation='relu'))

# Adding the third hidden layer
classifier.add(Dense(50, kernel_initializer = 'he_uniform',activation='relu'))
# Adding the output layer
classifier.add(Dense(1, kernel_initializer = 'he_uniform'))

# Compiling the ANN
classifier.compile(loss=root_mean_squared_error, optimizer='Adamax')

# Fitting the ANN to the Training set
model_history=classifier.fit(x_train.values, y_train.values,validation_split=0.20, batch_size = 10, epochs = 170)

Epoch 1/170
1403/1403 [==============================] - 2s 1ms/step - loss: 2.3849 - val_loss: 2.2250
Epoch 2/170
1403/1403 [==============================] - 2s 1ms/step - loss: 2.1236 - val_loss: 2.1547
Epoch 3/170
 611/1403 [============>.................] - ETA: 0s - loss: 2.049

KeyboardInterrupt: 

In [43]:
y_pred=classifier.predict(df1.values)

In [44]:
y_pred

array([[2.0068483],
       [6.7116036],
       [5.834105 ],
       ...,
       [6.3025017],
       [7.2923417],
       [6.379528 ]], dtype=float32)